In [1]:
import json

# The name of your JSON file
file_name = 'flavia.json'

try:
    # Open and load the JSON file
    with open(file_name, 'r') as f:
        data = json.load(f)

    # Loop through each query object in the list
    for query_obj in data:
        query_id = query_obj.get('query_id', 'N/A')
        description = query_obj.get('description', 'No description')
        
        print(f"--- Query ID: {query_id} ---")
        print(f"Description: {description}\n")

        # Loop through the questions for the current query
        for question in query_obj.get('questions', []):
            question_text = question.get('text', 'No question text')
            question_source = question.get('source', 'N/A')
            print(f"  - Question: {question_text}")
            print(f"    Source: {question_source}\n")

except FileNotFoundError:
    print(f"Error: The file '{file_name}' was not found.")
except json.JSONDecodeError:
    print(f"Error: The file '{file_name}' is not a valid JSON file.")

--- Query ID: DFLEXLIBS_001 ---
Description: Retrieves all available timeseries IDs related to HVAC zones, including various temperature setpoints, occupancy sensors, and VAV points.

  - Question: For each HVAC zone in the building, what are the timeseries IDs for the available zone temperature sensor' points, min and max temperature setpoints, temperature setpoints (single, heating/cooling or occupied/unoccupied), occupancy sensors and commands, and any related VAV damper, discharge temperature, and reheat command points?
    Source: human

  - Question: What is the name of each HVAC zone?
    Source: human

--- Query ID: DFLEXLIBS_002 ---
Description: Retrieves timeseries IDs for AHU supply air temperature, flow, and flow setpoint.

  - Question: For each AHU that supplies a zone, what are the timeseries IDs for all available points for supply air temperature, supply air flow, and supply air flow setpoint?
    Source: human

--- Query ID: DFLEXLIBS_003 ---
Description: Retrieves tim

In [12]:
# requirements: pip install rdflib

import rdflib

# requirements: pip install rdflib

import rdflib
from rdflib.namespace import RDFS

def parse_and_skolemize_ttl_files(file_paths):
    """
    Parses one or more TTL files into a single RDFLib Graph, replacing
    blank nodes (BNodes) with stable URIs (skolemization).

    Args:
        file_paths (list): A list of strings, where each string is the
                           path to a TTL file.

    Returns:
        rdflib.Graph: A graph object containing the parsed and skolemized
                      data from all files, or None if an error occurs.
    """
    # 1. Create a new Graph object. This will hold the combined data.
    g = rdflib.Graph()
    print("Graph created.")

    # 2. Iterate through the list of file paths provided.
    for file_path in file_paths:
        try:
            # 3. Parse each file into a temporary graph.
            print(f"Parsing {file_path}...")
            temp_g = rdflib.Graph()
            temp_g.parse(file_path, format='turtle')
            print(f"Successfully parsed {file_path}.")

            # 4. Skolemize the temporary graph to replace blank nodes.
            # This replaces BNodes with unique and valid URIs.
            # The base URI for the new nodes will be 'http://example.org/.well-known/genid/'
            temp_g.skolemize()
            
            # 5. Add the triples from the skolemized temp graph to the main graph.
            g += temp_g
            print(f"Skolemized and merged {file_path}.")

        except FileNotFoundError:
            print(f"Error: The file at {file_path} was not found.")
            return None
        except Exception as e:
            print(f"An error occurred while processing {file_path}: {e}")
            return None

    # 6. The graph `g` now contains all the triples from your files without BNodes.
    print(f"\nProcessing complete. The graph contains {len(g)} triples.")
    return g

if __name__ == '__main__':


    # List of the TTL files you want to parse.
    files_to_parse = ['Brick.ttl', 'brick-example.ttl']#['223p.ttl', 's223-example.ttl']
#['Brick (1).ttl', 'brick-example.ttl']#
    # Call the function to parse the files.
    brick_parsed = parse_and_skolemize_ttl_files(files_to_parse)

    if brick_parsed:
        print("\nGraph 'brick_parsed' has been created in memory.")

    #save the graph as a ttl file
    output_file = 'brick_parsed.ttl'
    try:
        brick_parsed.serialize(destination=output_file, format='turtle')
        print(f"\nGraph has been serialized and saved to {output_file}.")
    except Exception as e:
        print(f"An error occurred while saving the graph: {e}")





Graph created.
Parsing Brick.ttl...
Successfully parsed Brick.ttl.
Skolemized and merged Brick.ttl.
Parsing brick-example.ttl...
Successfully parsed brick-example.ttl.
Skolemized and merged brick-example.ttl.

Processing complete. The graph contains 57389 triples.

Graph 'brick_parsed' has been created in memory.

Graph has been serialized and saved to brick_parsed.ttl.


In [29]:
import rdflib
import pandas as pd
import json

# --- Helper Function ---
def extract_query_result_as_dataframe(sparql_result):
    var_names = [str(var) for var in sparql_result.vars]
    result_list = list(sparql_result)
    df = pd.DataFrame(result_list, columns=var_names)
    for column in df.columns:
        df[column] = df[column].apply(
            lambda x: x.split("#")[-1] if isinstance(x, rdflib.term.URIRef) else str(x) if x is not None else None
        )
    return df

# --- Configuration ---
ttl_file = "dflexlibs_multizone.ttl"
json_file = "dflexlibs_multizone.json"

# --- Load RDF Graph ---
print(f"\n--- Loading RDF data from '{ttl_file}' ---")
g = rdflib.Graph()
try:
    g.parse(ttl_file, format="turtle")
    print("✅ RDF data loaded successfully.\n")
except Exception as e:
    print(f"❌ Failed to load RDF file: {e}")
    exit()

# --- Load Query JSON ---
print(f"--- Loading queries from '{json_file}' ---")
try:
    with open(json_file, 'r') as f:
        all_building_data = json.load(f)
except Exception as e:
    print(f"❌ Failed to load JSON file: {e}")
    exit()

# --- Run Queries ---
for bldg_obj in all_building_data:
    queries = bldg_obj.get("queries", [])
    print(f"\n🏢 Processing {len(queries)} queries for building: {bldg_obj.get('building_id', 'unknown')}\n")
    
    for i, query_obj in enumerate(queries):
        query_id = query_obj.get("query_id", f"Query_{i}")
        sparql_query = query_obj.get("sparql_query")

        if not sparql_query:
            print(f"⚠️ Skipping {query_id}: No SPARQL query found.\n")
            continue

        print(f"▶️ Running {query_id}: {query_obj.get('description', '')}")
        try:
            result = g.query(sparql_query)
            result_list = list(result)
            print(f"   🔎 Found {len(result_list)} result(s)")

            if result_list:
                df = extract_query_result_as_dataframe(result)
                print(df.head())
            else:
                print("   ⚠️ No results returned.\n")

        except Exception as e:
            print(f"   ❌ Error running query {query_id}: {e}\n")



--- Loading RDF data from 'dflexlibs_multizone.ttl' ---
✅ RDF data loaded successfully.

--- Loading queries from 'dflexlibs_multizone.json' ---

🏢 Processing 6 queries for building: multizone_office_simple_air_with_data.ttl

▶️ Running DFLEXLIBS_001: BOPTEST building testcase_multizone_office_simple_air (Commercial building - 8500 m2)
   🔎 Found 1080 result(s)
       zone_name        zone_temp_point set_temp_min_point set_temp_max_point  \
0  hvac_cor_zone  hvac_reaZonCor_TZon_y     LowerSetp[cor]     UpperSetp[cor]   
1  hvac_cor_zone  hvac_reaZonCor_TZon_y     LowerSetp[cor]     UpperSetp[cor]   
2  hvac_cor_zone  hvac_reaZonCor_TZon_y     LowerSetp[cor]     UpperSetp[cor]   
3  hvac_cor_zone  hvac_reaZonCor_TZon_y     LowerSetp[cor]     UpperSetp[cor]   
4  hvac_cor_zone  hvac_reaZonCor_TZon_y     LowerSetp[cor]     UpperSetp[cor]   

  occ_sensor_point occ_cmd_point        zone_set_temp_heat_point  \
0   Occupancy[cor]          None  hvac_oveZonSupCor_TZonHeaSet_u   
1   Occupanc

In [12]:
import os
import shutil
import rdflib

def update_ttl_files(input_dir, output_dir):
    """
    Reads all TTL files from an input directory, replaces a specific string,
    and saves them to an output directory.

    Args:
        input_dir (str): The path to the directory containing the original
                         TTL files.
        output_dir (str): The path to the directory where the updated TTL
                          files will be saved.
    """
    # --- Create Output Directory ---
    if os.path.exists(output_dir):
        print(f"Output directory '{output_dir}' already exists. Clearing it before proceeding.")
        shutil.rmtree(output_dir)
    os.makedirs(output_dir)
    print(f"Successfully created or cleared output directory: '{output_dir}'")

    # --- Process and Replace String in Files ---
    try:
        files_processed = 0
        print("\n--- Starting File Content Replacement ---")
        for filename in os.listdir(input_dir):
            if filename.endswith(".ttl"):
                input_file_path = os.path.join(input_dir, filename)
                output_file_path = os.path.join(output_dir, filename)

                print(f"Processing '{input_file_path}'...")
                try:
                    with open(input_file_path, 'r', encoding='utf-8') as f:
                        content = f.read()
                    
                    updated_content = content.replace("Air_Handler_Unit", "Air_Handling_Unit")

                    with open(output_file_path, 'w', encoding='utf-8') as f:
                        f.write(updated_content)
                    
                    files_processed += 1
                    print(f" -> Saved updated file to '{output_file_path}'")
                except Exception as e:
                    print(f"Error processing file {filename}: {e}")
        
        if files_processed == 0:
            print(f"\nNo '.ttl' files were found in '{input_dir}'.")
        else:
            print(f"\nFinished processing. A total of {files_processed} TTL files were updated.")
    
    except FileNotFoundError:
        print(f"Error: The input directory '{input_dir}' was not found.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def test_updated_buildings(directory, sparql_query):
    """
    Loads each TTL file from a directory and runs a SPARQL query,
    reporting which files return results.

    Args:
        directory (str): The directory containing the TTL files to test.
        sparql_query (str): The SPARQL query to execute.
    """
    print("\n--- Starting SPARQL Query Test on Updated Files ---")
    buildings_with_results = []
    
    try:
        for filename in os.listdir(directory):
            if filename.endswith(".ttl"):
                file_path = os.path.join(directory, filename)
                print(f"Testing '{filename}'...")

                try:
                    g = rdflib.Graph()
                    g.parse(file_path, format='turtle')
                    
                    results = g.query(sparql_query)
                    
                    if len(results) > 0:
                        print(f"  --> SUCCESS: Found {len(results)} results in '{filename}'.")
                        buildings_with_results.append(filename)
                    else:
                        print(f"  --> No results found in '{filename}'.")

                except Exception as e:
                    print(f"  --> ERROR: Could not parse or query '{filename}'. Reason: {e}")
        
        print("\n--- Test Summary ---")
        if buildings_with_results:
            print("The following buildings returned a result for the query:")
            for building in buildings_with_results:
                print(f"- {building}")
        else:
            print("No buildings returned a result for the specified SPARQL query.")

    except FileNotFoundError:
        print(f"Error: The directory '{directory}' was not found for testing.")
    except Exception as e:
        print(f"An unexpected error occurred during testing: {e}")


# --- Configuration ---
input_directory = "mortargraphs_updated"
output_directory = "mortar_updated2"

SPARQL_QUERY_TO_TEST = """
    SELECT ?cooling_point ?heating_point ?ahu WHERE {
        ?ahu rdf:type/rdfs:subClassOf* brick:Air_Handling_Unit .
        ?cooling_point rdf:type/rdfs:subClassOf* brick:Cooling_Command .
        ?heating_point rdf:type/rdfs:subClassOf* brick:Heating_Command .
        ?ahu brick:hasPoint ?cooling_point .
        ?ahu brick:hasPoint ?heating_point .
    }
"""
# --- End Configuration ---

# --- Execution ---
if __name__ == "__main__":
    if not os.path.isdir(input_directory):
         print(f"Error: Input directory '{input_directory}' not found.")
         print("Please create it and place your TTL files inside.")
    else:
        # Step 1: Update the TTL files
        update_ttl_files(input_directory, output_directory)
        
        # Step 2: Test the newly created files with the SPARQL query
        test_updated_buildings(output_directory, SPARQL_QUERY_TO_TEST)


Successfully created or cleared output directory: 'mortar_updated2'

--- Starting File Content Replacement ---
Processing 'mortargraphs_updated/bldg5.ttl'...
 -> Saved updated file to 'mortar_updated2/bldg5.ttl'
Processing 'mortargraphs_updated/bldg4.ttl'...
 -> Saved updated file to 'mortar_updated2/bldg4.ttl'
Processing 'mortargraphs_updated/bldg6.ttl'...
 -> Saved updated file to 'mortar_updated2/bldg6.ttl'
Processing 'mortargraphs_updated/bldg44.ttl'...
 -> Saved updated file to 'mortar_updated2/bldg44.ttl'
Processing 'mortargraphs_updated/bldg7.ttl'...
 -> Saved updated file to 'mortar_updated2/bldg7.ttl'
Processing 'mortargraphs_updated/bldg3.ttl'...
 -> Saved updated file to 'mortar_updated2/bldg3.ttl'
Processing 'mortargraphs_updated/bldg41.ttl'...
 -> Saved updated file to 'mortar_updated2/bldg41.ttl'
Processing 'mortargraphs_updated/bldg40.ttl'...
 -> Saved updated file to 'mortar_updated2/bldg40.ttl'
Processing 'mortargraphs_updated/bldg2.ttl'...
 -> Saved updated file to 'm

In [6]:
# You must have the required libraries installed:
# pip install rdflib pandas numpy

import os
import rdflib
import pandas as pd
import numpy as np

# --- Configuration ---
graphs_directory = "mortargraphs_updated"
# --- End Configuration ---

file_triple_counts = []

print(f"--- Analyzing Triple Counts in '{graphs_directory}' ---")

# Get a list of all files in the specified directory and filter for .ttl files
all_files = os.listdir(graphs_directory)
ttl_files = [f for f in all_files if f.endswith('.ttl')]

print(f"Found {len(ttl_files)} TTL files to process...")

# Loop through each .ttl file
for ttl_filename in ttl_files:
    file_path = os.path.join(graphs_directory, ttl_filename)
    print(f"  - Processing '{ttl_filename}'...")
    g = rdflib.Graph()
    g.parse(file_path, format='turtle')
    num_triples = len(g)
    file_triple_counts.append({'filename': ttl_filename, 'triples': num_triples})

# After processing all files, analyze the results
# Convert the list of dictionaries to a pandas DataFrame for easy analysis
counts_df = pd.DataFrame(file_triple_counts)

# Calculate min, median, and max values
min_triples = counts_df['triples'].min()
median_triples = counts_df['triples'].median()
max_triples = counts_df['triples'].max()

# Find the corresponding filenames
min_files = counts_df[counts_df['triples'] == min_triples]['filename'].tolist()
max_files = counts_df[counts_df['triples'] == max_triples]['filename'].tolist()

# For the median, find the file(s) with the triple count closest to the median value
counts_df['median_diff'] = abs(counts_df['triples'] - median_triples)
min_median_diff = counts_df['median_diff'].min()
median_files = counts_df[counts_df['median_diff'] == min_median_diff]['filename'].tolist()

print("\n--- Analysis Results ---")
print(f"Minimum Triples: {min_triples} (File(s): {', '.join(min_files)})")
print(f"Median Triples:  {median_triples:.0f} (Closest File(s): {', '.join(median_files)})")
print(f"Maximum Triples: {max_triples} (File(s): {', '.join(max_files)})")

print("\n--- Full Data (sorted by triple count) ---")
# Set pandas to display all rows for better viewing
pd.set_option('display.max_rows', None)
print(counts_df[['filename', 'triples']].sort_values(by='triples').reset_index(drop=True))
# Reset the display option if desired
pd.reset_option('display.max_rows')


--- Analyzing Triple Counts in 'mortargraphs_updated' ---
Found 44 TTL files to process...
  - Processing 'bldg5.ttl'...
  - Processing 'bldg4.ttl'...
  - Processing 'bldg6.ttl'...
  - Processing 'bldg44.ttl'...
  - Processing 'bldg7.ttl'...
  - Processing 'bldg3.ttl'...
  - Processing 'bldg41.ttl'...
  - Processing 'bldg40.ttl'...
  - Processing 'bldg2.ttl'...
  - Processing 'bldg42.ttl'...
  - Processing 'bldg43.ttl'...
  - Processing 'bldg1.ttl'...
  - Processing 'bldg18.ttl'...
  - Processing 'bldg24.ttl'...
  - Processing 'bldg31.ttl'...
  - Processing 'bldg25.ttl'...
  - Processing 'bldg19.ttl'...
  - Processing 'bldg33.ttl'...
  - Processing 'bldg27.ttl'...
  - Processing 'bldg26.ttl'...
  - Processing 'bldg32.ttl'...
  - Processing 'smc.ttl'...
  - Processing 'bldg36.ttl'...
  - Processing 'bldg22.ttl'...
  - Processing 'bldg23.ttl'...
  - Processing 'bldg37.ttl'...
  - Processing 'bldg21.ttl'...
  - Processing 'bldg35.ttl'...
  - Processing 'bldg34.ttl'...
  - Processing 'bldg

--- Step 1: Analyzing Triple Counts in 'mortar_bricked' ---
Analyzed 44 building graphs.

--- Step 2: Loading queries from 'mortar.json' ---
Loaded 9 queries.

--- Step 3: Finding the most successful buildings ---


KeyboardInterrupt: 

In [13]:
# You must have the required libraries installed:
# pip install rdflib

import os
import rdflib

# --- Configuration ---
source_directory = "mortar_updated2"
brick_file_path = "Brick.ttl" 
destination_directory = "mortar_bricked"
# --- End Configuration ---

# 1. Create the destination directory if it doesn't exist
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)
    print(f"Created directory: '{destination_directory}'")

# 2. Load the Brick graph into memory once for efficiency
print(f"Loading '{brick_file_path}' into memory...")
brick_graph = rdflib.Graph()
try:
    brick_graph.parse(brick_file_path, format='turtle')
    print("Brick graph loaded successfully.")
except FileNotFoundError:
    print(f"Error: The file '{brick_file_path}' was not found. Please ensure it's in the correct location.")
    # Exit the script if Brick.ttl is not found
    exit()
except Exception as e:
    print(f"An error occurred parsing '{brick_file_path}': {e}")
    exit()


# 3. Get a list of all .ttl files from the source directory
try:
    all_files = os.listdir(source_directory)
    ttl_files = [f for f in all_files if f.endswith('.ttl')]
    print(f"\nFound {len(ttl_files)} building graphs to process in '{source_directory}'.")
except FileNotFoundError:
    print(f"Error: The source directory '{source_directory}' was not found.")
    exit()

# 4. Loop through each building graph, merge with Brick, and save
print("\n--- Starting merge process ---")
for ttl_filename in ttl_files:
    source_file_path = os.path.join(source_directory, ttl_filename)
    destination_file_path = os.path.join(destination_directory, ttl_filename)
    
    print(f"  - Processing '{ttl_filename}'...")
    
    # Create a new graph for the building data
    building_graph = rdflib.Graph()
    
    # Parse the building's TTL file
    building_graph.parse(source_file_path, format='turtle')
    
    # Merge the building graph with the pre-loaded Brick graph
    # The '+' operator in rdflib combines the two graphs
    combined_graph = building_graph + brick_graph
    
    # Serialize and save the combined graph to the new directory
    combined_graph.serialize(destination=destination_file_path, format='turtle')
    print(f"    -> Saved combined graph to '{destination_file_path}'")

print("\n--- Process complete ---")



Created directory: 'mortar_bricked'
Loading 'Brick.ttl' into memory...
Brick graph loaded successfully.

Found 44 building graphs to process in 'mortar_updated2'.

--- Starting merge process ---
  - Processing 'bldg5.ttl'...
    -> Saved combined graph to 'mortar_bricked/bldg5.ttl'
  - Processing 'bldg4.ttl'...
    -> Saved combined graph to 'mortar_bricked/bldg4.ttl'
  - Processing 'bldg6.ttl'...
    -> Saved combined graph to 'mortar_bricked/bldg6.ttl'
  - Processing 'bldg44.ttl'...
    -> Saved combined graph to 'mortar_bricked/bldg44.ttl'
  - Processing 'bldg7.ttl'...
    -> Saved combined graph to 'mortar_bricked/bldg7.ttl'
  - Processing 'bldg3.ttl'...
    -> Saved combined graph to 'mortar_bricked/bldg3.ttl'
  - Processing 'bldg41.ttl'...
    -> Saved combined graph to 'mortar_bricked/bldg41.ttl'
  - Processing 'bldg40.ttl'...
    -> Saved combined graph to 'mortar_bricked/bldg40.ttl'
  - Processing 'bldg2.ttl'...
    -> Saved combined graph to 'mortar_bricked/bldg2.ttl'
  - Pro

In [14]:
import os
import rdflib
import pandas as pd
import numpy as np
import json

# --- Configuration ---
graphs_directory = "mortar_bricked"
json_file = "mortar.json"
# --- End Configuration ---

# 1. Calculate triple counts for all TTL files for later use.
print(f"--- Step 1: Analyzing Triple Counts in '{graphs_directory}' ---")
file_triple_counts = []
all_files = os.listdir(graphs_directory)
ttl_files = [f for f in all_files if f.endswith('.ttl')]

for ttl_filename in ttl_files:
    file_path = os.path.join(graphs_directory, ttl_filename)
    g = rdflib.Graph()
    g.parse(file_path, format='turtle')
    file_triple_counts.append({'filename': ttl_filename, 'triples': len(g)})

counts_df = pd.DataFrame(file_triple_counts)
print(f"Analyzed {len(counts_df)} building graphs.\n")


# 2. Load the queries from the JSON file.
print(f"--- Step 2: Loading queries from '{json_file}' ---")
with open(json_file, 'r') as f:
    queries_data = json.load(f)
print(f"Loaded {len(queries_data)} queries.\n")


# 3. Find the buildings that are successful for the most queries.
print("--- Step 3: Finding the most successful buildings ---")
# Use a dictionary to keep track of how many queries each building answers.
building_success_counts = {filename: 0 for filename in ttl_files}

# Loop through each query
for query_info in queries_data:
    print(f"Processing query: {query_info.get('query_id', 'N/A')}")
    sparql_query = query_info['sparql_query']
    
    # Then loop through each building graph
    for ttl_filename in ttl_files:
        file_path = os.path.join(graphs_directory, ttl_filename)
        g = rdflib.Graph()
        g.parse(file_path, format='turtle')
        
        try:
            results = g.query(sparql_query)
            if len(results) > 0:
                # If the query returns results, increment the counter for that building
                building_success_counts[ttl_filename] += 1
        except Exception as e:
            # In case a SPARQL query has a syntax error
            print(f"Could not execute query {query_info.get('query_id', 'N/A')} on {ttl_filename}: {e}")

# Find the highest number of successful queries
max_success_count = 0
if building_success_counts:
    max_success_count = max(building_success_counts.values())

# Identify all buildings that have this maximum success count
top_buildings = [building for building, count in building_success_counts.items()
                 if count == max_success_count]

# --- Final Analysis ---
print(f"\n--- Step 4: Analysis of Top-Performing Buildings ---")

if not top_buildings:
    print("No buildings returned results for any of the queries.")
else:
    # Print the primary result: the most successful buildings
    print(f" बिल्डिंग(s) that returned results for the most SPARQL queries ({max_success_count} queries) are:")
    for building in top_buildings:
        print(f"  - {building}")

    # Now, get the triple counts for just these top buildings
    top_buildings_df = counts_df[counts_df['filename'].isin(top_buildings)]
    
    # Calculate and print the required statistics
    min_triples = top_buildings_df['triples'].min()
    median_triples = top_buildings_df['triples'].median()
    max_triples = top_buildings_df['triples'].max()

    print("\n--- Triple Count Statistics for These Top Buildings ---")
    print(f"  Minimum Triples: {min_triples}")
    print(f"  Median Triples:  {int(median_triples)}")
    print(f"  Maximum Triples: {max_triples}")
    print("-------------------------------------------------------")

--- Step 1: Analyzing Triple Counts in 'mortar_bricked' ---
Analyzed 44 building graphs.

--- Step 2: Loading queries from 'mortar.json' ---
Loaded 9 queries.

--- Step 3: Finding the most successful buildings ---
Processing query: MORTAR_001
Processing query: MORTAR_002
Processing query: MORTAR_003
Processing query: MORTAR_004
Processing query: MORTAR_005
Processing query: MORTAR_006
Processing query: MORTAR_007
Processing query: MORTAR_008
Processing query: MORTAR_009

--- Step 4: Analysis of Top-Performing Buildings ---
 बिल्डिंग(s) that returned results for the most SPARQL queries (9 queries) are:
  - bldg5.ttl
  - bldg4.ttl
  - bldg44.ttl
  - bldg40.ttl
  - bldg18.ttl
  - bldg39.ttl
  - bldg11.ttl
  - bldg13.ttl

--- Triple Count Statistics for These Top Buildings ---
  Minimum Triples: 54408
  Median Triples:  55516
  Maximum Triples: 62578
-------------------------------------------------------


In [25]:
import os
import rdflib

def run_building_tests(buildings_to_test, queries_to_run, ttl_directory="."):
    """
    Tests a list of buildings against a hardcoded list of SPARQL queries.

    For each building, it loads the TTL graph and executes all provided 
    queries against it, reporting the results.

    Args:
        buildings_to_test (list): A list of building names (e.g., "bestest_air_with_data").
        queries_to_run (list): A list of SPARQL query objects (dictionaries).
        ttl_directory (str): The directory where the .ttl files are located.
    """
    print(f"--- Starting Tests on {len(buildings_to_test)} Buildings ---")
    
    for building_name in buildings_to_test:
        building_ttl_file = f"{building_name}.ttl"
        building_path = os.path.join(ttl_directory, building_ttl_file)
        
        print(f"\n--- Processing Building: {building_ttl_file} ---")

        # --- Load Graph and Count Triples ---
        try:
            g = rdflib.Graph()
            g.parse(building_path, format='turtle')
            num_triples = len(g)
            print(f"Graph loaded successfully. Found {num_triples} triples.")
        except FileNotFoundError:
            print(f"Error: TTL file not found at '{building_path}'. Skipping.")
            continue
        except Exception as e:
            print(f"Error parsing '{building_path}': {e}. Skipping.")
            continue

        # --- Execute Applicable Queries ---
        successful_query_count = 0
        for query_obj in queries_to_run:
            query_id = query_obj.get('query_id')
            sparql_query = query_obj.get('sparql_query')
            
            try:
                results = g.query(sparql_query)
                if len(results) > 0:
                    successful_query_count += 1
                    print(f"  - SUCCESS: {query_id} returned {len(results)} results.")
                else:
                    print(f"  - NO RESULT: {query_id}")
            except Exception as e:
                print(f"  - ERROR executing {query_id}: {e}")

        # --- Report Summary for the Building ---
        print(f"\n* Summary for {building_ttl_file}:")
        print(f"  - Total Triples: {num_triples}")
        print(f"  - Successful Queries: {successful_query_count} / {len(queries_to_run)}")
        print("-" * 35)

# --- Configuration ---
# List of buildings to test (must match the filenames)
BUILDINGS_TO_TEST = [
    "bestest_air_with_data",
    "bestest_hydronic_heat_pump_with_data",
    "multizone_office_simple_air_with_data",
    "singlezone_commercial_hydronic_with_data"
]

# Hardcoded list of SPARQL queries
QUERIES = [
    {
        "query_id": "DFLEXLIBS_001_Zone_Info",
        "sparql_query": """
SELECT ?zone_name ?zone_temp_point ?set_temp_min_point ?set_temp_max_point ?occ_sensor_point ?occ_cmd_point 
?zone_set_temp_heat_point  ?zone_set_temp_cool_point ?unocc_zone_set_temp_heat_point ?unocc_zone_set_temp_cool_point 
?occ_zone_set_temp_heat_point ?occ_zone_set_temp_cool_point  ?zone_set_temp_point ?vav_damper_set_point ?vav_discharge_temp_point 
?vav_reheat_command_point

WHERE {
    ?zone a brick:HVAC_Zone  ;
    rdfs:label ?zone_name.
   
    OPTIONAL { 
        ?zone     brick:hasPoint ?TZon.
        ?TZon a brick:Zone_Air_Temperature_Sensor ;
            ref:hasExternalReference/ref:hasTimeseriesId ?zone_temp_point .}
    
    OPTIONAL {
        ?zone brick:hasPoint ?TSetMin .
        ?TSetMin a brick:Min_Air_Temperature_Setpoint ;
                ref:hasExternalReference/ref:hasTimeseriesId ?set_temp_min_point .}

    OPTIONAL {
        ?zone brick:hasPoint ?TSetMax .
        ?TSetMax a brick:Max_Air_Temperature_Setpoint ;
                    ref:hasExternalReference/ref:hasTimeseriesId ?set_temp_max_point .}
    
    OPTIONAL {
        ?zone brick:hasPoint ?TSetHeaZon .
        ?TSetHeaZon a brick:Zone_Air_Heating_Temperature_Setpoint;
                    ref:hasExternalReference/ref:hasTimeseriesId ?zone_set_temp_heat_point .}

    OPTIONAL {
        ?zone brick:hasPoint ?TSetCooZon .
        ?TSetCooZon a brick:Zone_Air_Cooling_Temperature_Setpoint;
                    ref:hasExternalReference/ref:hasTimeseriesId ?zone_set_temp_cool_point .}


    OPTIONAL {
        ?zone brick:hasPoint ?OccSensor.
        ?OccSensor a brick:Occupancy_Sensor ;
                   ref:hasExternalReference/ref:hasTimeseriesId ?occ_sensor_point .}

    OPTIONAL {
        ?zone brick:hasPoint ?OccCmd .
        ?OccCmd a brick:Occupancy_Command ;
                ref:hasExternalReference/ref:hasTimeseriesId ?occ_cmd_point .}
    
    OPTIONAL { 
         ?zone     brick:hasPoint ?UnOccTSetCooZon.
        ?UnOccTSetCooZon a brick:Unoccupied_Cooling_Temperature_Setpoint ;
                          ref:hasExternalReference/ref:hasTimeseriesId ?unocc_zone_set_temp_cool_point . 
    }
    
    OPTIONAL {
        ?zone brick:hasPoint ?UnOccTSetHeaZon .
        ?UnOccTSetHeaZon a brick:Unoccupied_Heating_Temperature_Setpoint;
                         ref:hasExternalReference/ref:hasTimeseriesId ?unocc_zone_set_temp_heat_point .}
    OPTIONAL {
        ?zone brick:hasPoint ?OccTSetCooZon .
        ?OccTSetCooZon a brick:Occupied_Cooling_Temperature_Setpoint;
                       ref:hasExternalReference/ref:hasTimeseriesId ?occ_zone_set_temp_cool_point .}

    OPTIONAL { 
        ?zone brick:hasPoint ?OccTSetHeaZon . 
        ?OccTSetHeaZon a brick:Occupied_Heating_Temperature_Setpoint ;
                        ref:hasExternalReference/ref:hasTimeseriesId ?occ_zone_set_temp_heat_point . 
    }
    
    OPTIONAL {
        ?zone brick:hasPoint ?TSetZon .
        ?TSetZon a brick:Zone_Air_Temperature_Setpoint;
                 ref:hasExternalReference/ref:hasTimeseriesId ?zone_set_temp_point .}
    
    OPTIONAL {
        ?vav brick:feeds ?zone ;
             brick:hasPoint ?Vav_DamSet .
        ?Vav_DamSet a brick:Damper_Position_Setpoint;
                    ref:hasExternalReference/ref:hasTimeseriesId ?vav_damper_set_point .}

    OPTIONAL {
        ?vav brick:feeds ?zone ;
             brick:hasPoint ?Vav_DisTemp .
        ?Vav_DisTemp a brick:Discharge_Air_Temperature_Sensor;
                     ref:hasExternalReference/ref:hasTimeseriesId ?vav_discharge_temp_point .}
    
    OPTIONAL {
        ?vav brick:feeds ?zone ;
             brick:hasPoint ?Vav_ReheatCmd .
        ?Vav_ReheatCmd a brick:Valve_Command;
                       ref:hasExternalReference/ref:hasTimeseriesId ?vav_reheat_command_point .}


}    
        """
    },
    {
        "query_id": "DFLEXLIBS_002_AHU_Info",
        "sparql_query": """
SELECT ?ahu_supply_temp_point ?ahu_supply_flow_point ?ahu_supply_flow_set_point
WHERE {
    
    OPTIONAL {
    ?ahu brick:feeds/brick:feeds ?zone ;
         brick:hasPoint ?Ahu_TSup .
    ?Ahu_TSup a brick:Supply_Air_Temperature_Sensor;
        ref:hasExternalReference/ref:hasTimeseriesId ?ahu_supply_temp_point .}

    OPTIONAL {
    ?ahu brick:feeds/brick:feeds ?zone ;
         brick:hasPoint ?Ahu_flowSup .
    ?Ahu_flowSup a brick:Supply_Air_Flow_Sensor;
        ref:hasExternalReference/ref:hasTimeseriesId ?ahu_supply_flow_point .}

    OPTIONAL {
    ?ahu brick:feeds/brick:feeds ?zone ;
         brick:hasPoint ?Ahu_flowSupSet .
    ?Ahu_flowSupSet a brick:Supply_Air_Flow_Setpoint;
        ref:hasExternalReference/ref:hasTimeseriesId ?ahu_supply_flow_set_point .}

}
        """
    },
    {
        "query_id": "DFLEXLIBS_003_Power_Sensors",
        "sparql_query": """
SELECT DISTINCT ?ele_pow_point ?therm_pow_point
WHERE {
    
        OPTIONAL {
    ?Epower a brick:Electric_Power_Sensor ;
               ref:hasExternalReference/ref:hasTimeseriesId ?ele_pow_point .}

    OPTIONAL {
    ?Tpower a brick:Thermal_Power_Sensor ;
               ref:hasExternalReference/ref:hasTimeseriesId ?therm_pow_point .}


}
        """
    },
    {
        "query_id": "DFLEXLIBS_004_Building_Area",
        "sparql_query": """
SELECT ?area WHERE {
    ?building a brick:Building ;
    brick:area [brick:value ?area] .

}
        """
    },
    {
        "query_id": "DFLEXLIBS_005_Electric_Power_Baseline",
        "sparql_query": """
            PREFIX brick: <https://brickschema.org/schema/Brick#>
            PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
            SELECT ?power_point WHERE {
                ?point a brick:Electric_Power_Sensor ;
                ref:hasExternalReference ?ref .
                ?ref ref:onTable "baseline_" ;
                ref:hasTimeseriesId ?power_point .
            }
        """
    },
    {
        "query_id": "DFLEXLIBS_005_Electric_Power_Flexible",
        "sparql_query": """
            PREFIX brick: <https://brickschema.org/schema/Brick#>
            PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
            SELECT ?power_point WHERE {
                ?point a brick:Electric_Power_Sensor ;
                ref:hasExternalReference ?ref .
                ?ref ref:onTable "flexible_" ;
                ref:hasTimeseriesId ?power_point .
            }
        """
    },
    {
        "query_id": "DFLEXLIBS_006_Thermal_Power_Baseline",
        "sparql_query": """
            PREFIX brick: <https://brickschema.org/schema/Brick#>
            PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
            SELECT ?power_point WHERE {
                ?point a brick:Thermal_Power_Sensor ;
                ref:hasExternalReference ?ref .
                ?ref ref:onTable "baseline_" ;
                ref:hasTimeseriesId ?power_point .
            }
        """
    },
    {
        "query_id": "DFLEXLIBS_006_Thermal_Power_Flexible",
        "sparql_query": """
            PREFIX brick: <https://brickschema.org/schema/Brick#>
            PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
            SELECT ?power_point WHERE {
                ?point a brick:Thermal_Power_Sensor ;
                ref:hasExternalReference ?ref .
                ?ref ref:onTable "flexible_" ;
                ref:hasTimeseriesId ?power_point .
            }
        """
    }
]

# Directory containing the .ttl files for the buildings.
# Assumes they are in the same directory as the notebook.
# Change this if your files are elsewhere, e.g., "path/to/your/ttl/files"
TTL_FILES_DIRECTORY = "." 

# --- Execution ---
run_building_tests(BUILDINGS_TO_TEST, QUERIES, TTL_FILES_DIRECTORY)


--- Starting Tests on 4 Buildings ---

--- Processing Building: bestest_air_with_data.ttl ---
Graph loaded successfully. Found 132 triples.
  - SUCCESS: DFLEXLIBS_001_Zone_Info returned 216 results.
  - SUCCESS: DFLEXLIBS_002_AHU_Info returned 1 results.
  - SUCCESS: DFLEXLIBS_003_Power_Sensors returned 2 results.
  - SUCCESS: DFLEXLIBS_004_Building_Area returned 1 results.
  - NO RESULT: DFLEXLIBS_005_Electric_Power_Baseline
  - NO RESULT: DFLEXLIBS_005_Electric_Power_Flexible
  - NO RESULT: DFLEXLIBS_006_Thermal_Power_Baseline
  - NO RESULT: DFLEXLIBS_006_Thermal_Power_Flexible

* Summary for bestest_air_with_data.ttl:
  - Total Triples: 132
  - Successful Queries: 4 / 8
-----------------------------------

--- Processing Building: bestest_hydronic_heat_pump_with_data.ttl ---
Graph loaded successfully. Found 102 triples.
  - SUCCESS: DFLEXLIBS_001_Zone_Info returned 72 results.
  - SUCCESS: DFLEXLIBS_002_AHU_Info returned 1 results.
  - SUCCESS: DFLEXLIBS_003_Power_Sensors returned 3

Processing 'dflexlibs_multizone.json'...
Reading generated questions from 'dflexlibs_multizone_generated_questions.csv'...
Successfully merged questions.
New file saved to 'combined_data/dflexlibs_multizone_combined.json'
